In [3]:
from TMfunctions import *
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
nltk.download('stopwords')
import re
import networkx
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from Bio import Entrez
from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [5]:
from sklearn.model_selection import KFold

# Separe Titles from Abstracts

In [2]:
train = pd.read_csv('Data/No PreProcessing/train.csv', index_col = 0)
test = pd.read_csv('Data/No PreProcessing/test.csv', index_col = 0)
data = pd.concat([train, test])

In [3]:
def search(query):
    Entrez.email = 'd.giardini2@campus.unimib.it'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='1',
                            retmode='xml',
                            term=query,
                           field='title',
                           api_key='384b5a209a2bb99c452bd22b2f0b2add3f09')
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'd.giardini2@campus.unimib.it'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [4]:
import pickle

In [19]:
errors = []
for index, row in tqdm(data.iterrows()):
    if index in list(diz.keys()):
        continue
    ids = search(row['medical_abstract'].split('.')[0])['IdList']
    if not ids:
        errors.append(index)
        continue
    result = fetch_details(ids)
    try:
        diz[index] = result['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
    except:
        errors.append(index)
        continue
    with open('Data/titles.pkl', 'wb') as f:
        pickle.dump(diz, f)
    time.sleep(0.2)

9445it [3:43:16,  1.42s/it]


In [49]:
with open('Data/errors.pkl', 'wb') as f:
        pickle.dump(errors, f)

In [48]:
len(errors)

71

## Build Dataset

In [50]:
train = pd.read_csv('Data/No PreProcessing/train.csv', index_col = 0)
test = pd.read_csv('Data/No PreProcessing/test.csv', index_col = 0)
data = pd.concat([train, test])

In [64]:
lst = []
for key,value in tqdm(titles.items()):
    diz = {'col_id' : key, 'title' : value}
    tit_abst = data.loc[key, 'medical_abstract']
    abst = tit_abst.split(value)[-1].strip()
    diz['medical_abstract'] = abst
    lst.append(diz)

100%|███████████████████████████████████████████████████████████████████████████| 9403/9403 [00:00<00:00, 48740.90it/s]


In [72]:
df = pd.DataFrame(lst)
df.index = df['col_id']
df = df[['title', 'medical_abstract']]
df.head()

,title,medical_abstract
col_id,,
2839,Immunosuppressive acidic protein in malignant ...,The immunosuppressive acidic protein (IAP) has...
5670,Arterial mechanical properties in dilated card...,The effects of aging on arterial mechanical pr...
6062,Physiology of aging related to outcome in the ...,Thirty-nine patients with adult respiratory di...
5609,The long-distance effects of brain lesions: vi...,We describe several new possibilities for the ...
1924,Randomized phase II evaluation of carboplatin ...,A total of 83 patients with metastatic transit...


In [74]:
df.to_csv('Data/title&abstract.csv')

In [77]:
data = df

# Graph

In [75]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [79]:
stop_words = nltk.corpus.stopwords.words('english')
normalize_corpus = np.vectorize(normalize_document)

vectorizer = TfidfVectorizer(min_df=3, max_df=0.5)
vectorizer.fit(data['medical_abstract'])

TfidfVectorizer(max_df=0.5, min_df=3)

PageRank ranks each node based on a number of in-degree of that node, and inversely proportional to the out-degree of the other nodes that point to that node. In the proposed methodology, a network graph is built from Twitter where the user acts as a node and tweet-retweet relation as a directed edge. The user who retweet the tweet points to the original user who tweets. From the formed graph, each node’s PageRank is calculated as well as other node properties like centrality, degree, and followers, and average time retweeted. The result shows that the PageRank score of a node is directly proportional to closeness centrality and in-degree of the node. However, the ranking with PageRank, closeness centrality, and in-degree ranking yield different ranking results.

In [84]:
def summarize_doc(DOCUMENT, num_sentences=2):
    DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
    DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
    DOCUMENT = re.sub(r'\d+', ' ', DOCUMENT)
    pattern = r'\[[^\]]*\]'
    DOCUMENT = re.sub(pattern, ' ', DOCUMENT)
    DOCUMENT = DOCUMENT.strip()
    sentences = nltk.sent_tokenize(DOCUMENT)
    norm_sentences = normalize_corpus(sentences)
    norm_sentences[:3]
    dt_matrix = vectorizer.transform(norm_sentences)
    dt_matrix = dt_matrix.toarray()
    similarity_matrix = np.matmul(dt_matrix, dt_matrix.T)
    similarity_graph = networkx.from_numpy_array(similarity_matrix)
    scores = networkx.pagerank(similarity_graph)
    ranked_sentences = sorted(((score, index) for index, score  in scores.items()), reverse=True)
    try:
        top_sentence_indices = [ranked_sentences[index][1] for index in range(num_sentences)]
    except:
        top_sentence_indices = ranked_sentences[0][1]
        return ' '.join(np.array(sentences)[top_sentence_indices])
    top_sentence_indices.sort()
    return ' '.join(np.array(sentences)[top_sentence_indices])

In [85]:
data['summary'] = data['medical_abstract'].apply(summarize_doc)

In [87]:
pd.set_option('display.max_colwidth', None)
data[['medical_abstract', 'summary', 'title']]

,medical_abstract,summary,title
col_id,,,
2839,"The immunosuppressive acidic protein (IAP) has been described as a tumor associated marker in some solid tumors and hematologic diseases. To define the clinical relevance the authors determined the serum IAP levels in 194 patients with malignancies before initiation of therapy, 14 patients with idiopathic thrombocytopenic purpura (ITP), 28 patients with bacterial pneumonia, and 23 healthy volunteers. Immunosuppressive acidic protein was measured by radial immunodiffusion. The mean value of our controls was 405 +/- 48 micrograms/ml. This is consistent with published data. The mean values for patients with malignancies varied from 554 micrograms/ml to 698 micrograms/ml. These are only marginally higher than those observed for the controls. In contrast patients with bacterial pneumonia demonstrated significantly elevated values (1038 +/- 261 micrograms/ml). The authors conclude that IAP cannot be used as a diagnostic marker for the malignant diseases examined in this study.",The immunosuppressive acidic protein (IAP) has been described as a tumor associated marker in some solid tumors and hematologic diseases. The mean values for patients with malignancies varied from micrograms/ml to micrograms/ml.,Immunosuppressive acidic protein in malignant diseases. Clinical relevance?
5670,"The effects of aging on arterial mechanical properties and the response to nitroprusside were examined in 25 patients with dilated cardiomyopathy. High-fidelity pressures were recorded with a multisensor catheter. Pulse wave velocity was determined between two sensors in the thoracic aorta. Arterial compliance was determined by an analysis of the diastolic waveform and cardiac output. At baseline, despite a similar systemic vascular resistance, the pulsatile load (e.g., arterial compliance) and wave transmission characteristics (e.g., pulse wave velocity) were altered with aging. Arterial compliance was reduced in older (greater than 50 yr, n = 8) versus younger (less than 35 yr, n = 8) patients (0.51 +/- 0.17 vs. 1.33 +/- 0.63 ml/mmHg, P less than 0.01) and intermediate in those 35-50 yr of age (n = 9, 0.72 +/- 0.40 ml/mmHg). There was a positive correlation between age and pulse wave velocity (r = +0.90). Nitroprusside infusion decreased resistance, increased arterial compliance, and lowered pulse wave velocity in all groups. Yet, advancing age was associated with a greater fall in wave velocity for a given fall in aortic pressure. The slope (K) of the relation between pulse wave velocity and aortic diastolic pressure progressively increased with age (0.01 +/- 0.03, 0.06 +/- 0.02, and 0.09 +/- 0.03 m/s-mmHg). Multiple linear regression analysis revealed a significant relation between K and age. These data demonstrate that in older patients with dilated cardiomyopathy the left ventricle is coupled to an arterial circulation that has a greater pulsatile load, despite a similar steady load. Furthermore, these age-related changes in the arterial system affect the hemodynamic response to pharmacologically-induced vasodilatation.","There was a positive correlation between age and pulse wave velocity (r = + . Nitroprusside infusion decreased resistance, increased arterial compliance, and lowered pulse wave velocity in all groups.",Arterial mechanical properties in dilated cardiomyopathy. Aging and the response to nitroprusside.
6062,"Thirty-nine patients with adult respiratory distress syndrome (ARDS) were enrolled in a study to identify potential age-related changes in organ system function that may help explain the apparent association between age and poor outcome in these patients. Criteria for enrollment included an arterial PO2-to-inspired O2 concentration ratio less than or equal to 200 in a clinical setting consistent with ARDS. Patients were excluded if they were less than 18 yr old, had clinical manifestations of congestive heart failure, were seropositive for the human immunodeficiency virus, or had st

In [89]:
data.to_csv('Data/graphSumm.csv')

In [18]:
df = pd.read_csv('Data/graphSumm.csv')

In [20]:
df[df['summary'] == '']

,col_id,title,medical_abstract,summary


## Graph Evaluation

In [95]:
#from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')
candidates = list(data['summary'])

references = list(data['title'])
results = rouge.compute(predictions=candidates, references=references)
print(results)

{'rouge1': 0.21552359816270705, 'rouge2': 0.09005551600391185, 'rougeL': 0.17562077093867032, 'rougeLsum': 0.17561396384656563}


# LSA

In [5]:
data = pd.read_csv('Data/title&abstract.csv', index_col = 0)

In [26]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Apply preProcessing
def preprocess(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

for document in data['medical_abstract']:
    no_stop += [preprocess(x) for x in tokenizer.tokenize(document)]

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(no_stop)
X = tfidf_matrix.toarray()

In [30]:
X.shape

(72035, 31816)

In [31]:
Xt = np.transpose(X)

In [32]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(Xt)

len(svd_model.components_)

20

In [33]:
terms = tfidf.get_feature_names_out()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    string = "Topic "+str(i)+": "
    for t in sorted_terms:
        string += t[0].strip()
        string += ", "
    print(string)

Topic 0: hindquarters, cephamycins, stabilisations, nonmedical, lobes, impulse, henseleit, 
Topic 1: baroreceptors, strangulating, photosensitive, italy, angiotoxic, csmc, dermatographism, 
Topic 2: maltreated, pustules, restudied, met, ndga, hindquarters, ending, 
Topic 3: capsulothalamolenticular, meticulous, transposed, unspecified, flareups, strata, strategic, 
Topic 4: capsulothalamolenticular, meticulous, discusses, poliovirus, analgesics, slum, polyreactive, 
Topic 5: avenue, chylomicron, depiction, dissatisfied, engaged, evolving, graves, 
Topic 6: routine, overdrive, hms, csm, neovascularity, preferred, massive, 
Topic 7: doubles, repermeation, contamination, resorting, directors, persistently, granules, 
Topic 8: textural, italy, complainers, nebivolol, crescentic, rflps, turbid, 
Topic 9: ceramic, negotiate, posteriorly, adrenoreceptor, nephrogenesis, cryoglobulinemia, roseola, 
Topic 10: lacunae, leukomalacia, feeders, implied, stilbestrol, chaperone, thyronine, 
Topic 11: 

In [34]:
# Matrice V trasposta
Vt = svd_model.components_

In [35]:
Vt.shape

(20, 72035)

In [36]:
U = svd_model.fit_transform(Xt)

In [37]:
U.shape

(31816, 20)

In [38]:
sigma = np.diag(svd_model.singular_values_)

In [39]:
D = np.matmul(sigma, Vt)

In [40]:
D.shape

(20, 72035)

In [41]:
import pandas as pd

In [42]:
weights = np.power(pd.DataFrame(np.power(D,2)).sum(axis = 'rows'), 1/2)
weights

0        0.134329
1        0.242585
2        0.034525
3        0.277814
4        0.057175
           ...   
72030    0.101549
72031    0.157085
72032    0.182703
72033    0.040380
72034    0.047526
Length: 72035, dtype: float64

In [43]:
sentences = []
for document in data['medical_abstract']:
    sentences += [x for x in tokenizer.tokenize(document)]

In [45]:
df_weights = pd.DataFrame(weights)
df_weights.index = sentences

In [51]:
doc_sent = []
for document in data['medical_abstract']:
    doc_sent.append([x for x in tokenizer.tokenize(document)])

In [53]:
summaries = []
for document in doc_sent:
    best = list(df_weights.loc[document].sort_values(0, ascending = False).iloc[:2].index)
    summary = ' '.join([x for x in document if x in best])
    summaries.append(summary)

In [54]:
summary_df = data.copy()
summary_df['summary'] = summaries

In [55]:
pd.set_option('display.max_colwidth', None)
summary_df.head()

,title,medical_abstract,summary
col_id,,,
2839,Immunosuppressive acidic protein in malignant diseases. Clinical relevance?,"The immunosuppressive acidic protein (IAP) has been described as a tumor associated marker in some solid tumors and hematologic diseases. To define the clinical relevance the authors determined the serum IAP levels in 194 patients with malignancies before initiation of therapy, 14 patients with idiopathic thrombocytopenic purpura (ITP), 28 patients with bacterial pneumonia, and 23 healthy volunteers. Immunosuppressive acidic protein was measured by radial immunodiffusion. The mean value of our controls was 405 +/- 48 micrograms/ml. This is consistent with published data. The mean values for patients with malignancies varied from 554 micrograms/ml to 698 micrograms/ml. These are only marginally higher than those observed for the controls. In contrast patients with bacterial pneumonia demonstrated significantly elevated values (1038 +/- 261 micrograms/ml). The authors conclude that IAP cannot be used as a diagnostic marker for the malignant diseases examined in this study.","To define the clinical relevance the authors determined the serum IAP levels in 194 patients with malignancies before initiation of therapy, 14 patients with idiopathic thrombocytopenic purpura (ITP), 28 patients with bacterial pneumonia, and 23 healthy volunteers. The mean value of our controls was 405 +/- 48 micrograms/ml."
5670,Arterial mechanical properties in dilated cardiomyopathy. Aging and the response to nitroprusside.,"The effects of aging on arterial mechanical properties and the response to nitroprusside were examined in 25 patients with dilated cardiomyopathy. High-fidelity pressures were recorded with a multisensor catheter. Pulse wave velocity was determined between two sensors in the thoracic aorta. Arterial compliance was determined by an analysis of the diastolic waveform and cardiac output. At baseline, despite a similar systemic vascular resistance, the pulsatile load (e.g., arterial compliance) and wave transmission characteristics (e.g., pulse wave velocity) were altered with aging. Arterial compliance was reduced in older (greater than 50 yr, n = 8) versus younger (less than 35 yr, n = 8) patients (0.51 +/- 0.17 vs. 1.33 +/- 0.63 ml/mmHg, P less than 0.01) and intermediate in those 35-50 yr of age (n = 9, 0.72 +/- 0.40 ml/mmHg). There was a positive correlation between age and pulse wave velocity (r = +0.90). Nitroprusside infusion decreased resistance, increased arterial compliance, and lowered pulse wave velocity in all groups. Yet, advancing age was associated with a greater fall in wave velocity for a given fall in aortic pressure. The slope (K) of the relation between pulse wave velocity and aortic diastolic pressure progressively increased with age (0.01 +/- 0.03, 0.06 +/- 0.02, and 0.09 +/- 0.03 m/s-mmHg). Multiple linear regression analysis revealed a significant relation between K and age. These data demonstrate that in older patients with dilated cardiomyopathy the left ventricle is coupled to an arterial circulation that has a greater pulsatile load, despite a similar steady load. Furthermore, these age-related changes in the arterial system affect the hemodynamic response to pharmacologically-induced vasodilatation.","Arterial compliance was reduced in older (greater than 50 yr, n = 8) versus younger (less than 35 yr, n = 8) patients (0.51 +/- 0.17 vs. 1.33 +/- 0.63 ml/mmHg, P less than 0.01) and intermediate in those 35-50 yr of age (n = 9, 0.72 +/- 0.40 ml/mmHg). Multiple linear regression analysis revealed a significant relation between K and age."
6062,Physiology of aging related to outcome in the adult respiratory distress syndrome.,"Thirty-nine patients with adult respiratory distress syndrome (ARDS) were enrolled in a study to identify potential age-related changes in organ system function that may help explain the apparent association between age and poor outcome in these patients. Criteria for

In [56]:
summary_df.to_csv('Data/LSASumm.csv')

## LSA Evaluation

In [97]:
data = pd.read_csv('Data/LSASumm.csv')

In [98]:
#from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')
candidates = list(data['summary'])

references = list(data['title'])
results = rouge.compute(predictions=candidates, references=references)
print(results)

{'rouge1': 0.17958395781861622, 'rouge2': 0.06491358027816224, 'rougeL': 0.14147401955939062, 'rougeLsum': 0.14156975537796002}


# Casual Dataset

In [22]:
data = pd.read_csv('Data/title&abstract.csv', index_col = 0)

In [23]:
import random

In [24]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

rnd = []
for document in data['medical_abstract']:
    sentences = tokenizer.tokenize(document)
    if len(sentences) > 2:
        rnd.append(' '.join(random.sample(sentences, 2)))
    else:
        rnd.append(' '.join(sentences))

In [25]:
data['random_summary'] = rnd

## Random Evaluation

In [27]:
import evaluate
rouge = evaluate.load('rouge')
candidates = list(data['random_summary'])

references = list(data['title'])
results = rouge.compute(predictions=candidates, references=references)
print(results)

{'rouge1': 0.19889511104216673, 'rouge2': 0.07535874210351032, 'rougeL': 0.15696185120724238, 'rougeLsum': 0.15697829987334586}


In [28]:
data

,title,medical_abstract,random_summary
col_id,,,
2839,Immunosuppressive acidic protein in malignant ...,The immunosuppressive acidic protein (IAP) has...,These are only marginally higher than those ob...
5670,Arterial mechanical properties in dilated card...,The effects of aging on arterial mechanical pr...,"Yet, advancing age was associated with a great..."
6062,Physiology of aging related to outcome in the ...,Thirty-nine patients with adult respiratory di...,Criteria for enrollment included an arterial P...
5609,The long-distance effects of brain lesions: vi...,We describe several new possibilities for the ...,The methods are based on the visualization of ...
1924,Randomized phase II evaluation of carboplatin ...,A total of 83 patients with metastatic transit...,Independent factors prognostic for survival (p...
...,...,...,...
3125,Postoperative sore throat: topical hydrocortis...,Postoperative sore throat: topical hydrocortis...,They were allocated randomly into one of two g...
6430,Mortality from bladder cancer in dyestuff work...,"Cancer mortality in workers exposed to 2,3,7,8...",In the subcohort of 1520 workers with greater ...
4094,Advances in breast reconstruction after mastec...,Reconstruction after mastectomy. Advances in m...,Because of the psychological and cultural sign...


In [30]:
graph_df = pd.read_csv('Data/GraphSumm.csv', index_col = 0)

In [33]:
data = data.join(graph_df['summary'], how='left', on=None, sort=False)

In [36]:
data = data.rename(columns={"summary": "graph_summary"})

In [37]:
lsa_df = pd.read_csv('Data/LSASumm.csv', index_col = 0)
data = data.join(lsa_df['summary'], how='left', on=None, sort=False)
data = data.rename(columns={"summary": "lsa_summary"})

In [38]:
data.to_csv('Data/summaries.csv')

# Evaluation with Classification

In [5]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('Pretrained Models/PubMed-and-PMC-w2v.bin', binary=True)

C:\Users\david\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
df_summaries = pd.read_csv('Data/summaries.csv', index_col = 0)

In [14]:
train = pd.read_csv('Data/No PreProcessing/train.csv', index_col = 0)
test = pd.read_csv('Data/No PreProcessing/test.csv', index_col = 0)
data = pd.concat([train,test])

In [17]:
data = pd.merge(data[['1','2','3','4','5']], df_summaries, left_index=True, right_index=True)
#data = df_summaries.join(data[['1','2','3','4','5']], how='left', on=None, sort=False)
data.head()

,1,2,3,4,5,title,medical_abstract,random_summary,graph_summary,lsa_summary
2839,1,0,0,0,0,Immunosuppressive acidic protein in malignant ...,The immunosuppressive acidic protein (IAP) has...,These are only marginally higher than those ob...,The immunosuppressive acidic protein (IAP) has...,To define the clinical relevance the authors d...
5670,0,0,0,1,0,Arterial mechanical properties in dilated card...,The effects of aging on arterial mechanical pr...,"Yet, advancing age was associated with a great...",There was a positive correlation between age a...,Arterial compliance was reduced in older (grea...
6062,0,0,0,0,1,Physiology of aging related to outcome in the ...,Thirty-nine patients with adult respiratory di...,Criteria for enrollment included an arterial P...,Patients were divided into two groups: those l...,Mortality rates were 12% in the patients less ...
5609,0,0,1,0,1,The long-distance effects of brain lesions: vi...,We describe several new possibilities for the ...,The methods are based on the visualization of ...,"Even after lengthy formalin fixation, the meth...","Even after lengthy formalin fixation, the meth..."
1924,1,0,0,0,0,Randomized phase II evaluation of carboplatin ...,A total of 83 patients with metastatic transit...,Independent factors prognostic for survival (p...,Among evaluable patients with measurable disea...,Median survival for 64 evaluable patients was ...


In [19]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [model[word] for word in words if word in model]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [20]:
# Preprocess summary columns
def preprocess(text):
    # Remove punctuation and other non-alphanumeric characters
    text =  re.sub('[^a-zA-Z]', ' ', text)
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    # Remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # Join the words back into a string
    return ' '.join(words)

data['random_summary'] = data['random_summary'].apply(lambda x: preprocess(x))
data['graph_summary'] = data['graph_summary'].apply(lambda x: preprocess(x))
data['lsa_summary'] = data['lsa_summary'].apply(lambda x: preprocess(x))

In [22]:
data = data.drop(data[data['random_summary'] == ''].index)

In [23]:
data.to_csv('Data/summariesPreprocessed.csv')

In [24]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.iloc[train_index]
    test = data.iloc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['random_summary']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['random_summary']]))
    
    Kresult = micro_f1SVM(build_resultsSVM(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB, SVM = 0, 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
        SVM += result['SVM']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5, 'SVM': SVM/5}
result

fold 1:
{'DT': 0.451, 'RF': 0.516, 'NB': 0.626, 'SVM': 0.638}
 
fold 2:
{'DT': 0.476, 'RF': 0.517, 'NB': 0.604, 'SVM': 0.621}
 
fold 3:
{'DT': 0.478, 'RF': 0.511, 'NB': 0.614, 'SVM': 0.635}
 
fold 4:
{'DT': 0.472, 'RF': 0.515, 'NB': 0.629, 'SVM': 0.638}
 
fold 5:
{'DT': 0.472, 'RF': 0.505, 'NB': 0.625, 'SVM': 0.632}
 


{'DT': 0.46980000000000005,
 'RF': 0.5128,
 'NB': 0.6195999999999999,
 'SVM': 0.6328}

In [27]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.iloc[train_index]
    test = data.iloc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['graph_summary']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['graph_summary']]))
    
    Kresult = micro_f1SVM(build_resultsSVM(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB, SVM = 0, 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
        SVM += result['SVM']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5, 'SVM': SVM/5}
result

fold 1:
{'DT': 0.495, 'RF': 0.528, 'NB': 0.615, 'SVM': 0.651}
 
fold 2:
{'DT': 0.484, 'RF': 0.523, 'NB': 0.609, 'SVM': 0.647}
 
fold 3:
{'DT': 0.499, 'RF': 0.538, 'NB': 0.621, 'SVM': 0.653}
 
fold 4:
{'DT': 0.491, 'RF': 0.524, 'NB': 0.617, 'SVM': 0.639}
 
fold 5:
{'DT': 0.483, 'RF': 0.523, 'NB': 0.637, 'SVM': 0.64}
 


{'DT': 0.4904,
 'RF': 0.5272000000000001,
 'NB': 0.6197999999999999,
 'SVM': 0.646}

In [28]:
kf = KFold(n_splits=5)

fold = 0
Kresults = []

for train_index, test_index in kf.split(data):
    fold += 1
    train = data.iloc[train_index]
    test = data.iloc[test_index]
    
    FE_train = pd.DataFrame(np.array([vectorize(doc) for doc in train['lsa_summary']]))
    FE_test = pd.DataFrame(np.array([vectorize(doc) for doc in test['lsa_summary']]))
    
    Kresult = micro_f1SVM(build_resultsSVM(FE_train, FE_test, train, test))
    Kresults.append(Kresult)
    print("fold " + str(fold) + ":")
    print(Kresult)
    print(" ")
    
    DT, RF, NB, SVM = 0, 0, 0, 0
    for result in Kresults:
        DT += result['DT']
        RF += result['RF']
        NB += result['NB']
        SVM += result['SVM']
    result = {'DT': DT/5, 'RF': RF/5, 'NB': NB/5, 'SVM': SVM/5}
result

fold 1:
{'DT': 0.449, 'RF': 0.515, 'NB': 0.596, 'SVM': 0.626}
 
fold 2:
{'DT': 0.485, 'RF': 0.505, 'NB': 0.586, 'SVM': 0.622}
 
fold 3:
{'DT': 0.473, 'RF': 0.519, 'NB': 0.597, 'SVM': 0.643}
 
fold 4:
{'DT': 0.482, 'RF': 0.511, 'NB': 0.588, 'SVM': 0.63}
 
fold 5:
{'DT': 0.467, 'RF': 0.507, 'NB': 0.604, 'SVM': 0.62}
 


{'DT': 0.47119999999999995,
 'RF': 0.5114000000000001,
 'NB': 0.5942000000000001,
 'SVM': 0.6282}